# How to use the exposed factory functions for Matrix types

In [ ]:
import numpy as np
import ipywidgets as widgets
from matplotlib import pyplot as plt
from IPython.display import display

from lmath import Vector2f, Vector3f, Vector4f
from lmath import Vector2d, Vector3d, Vector4d
from lmath import Matrix2f, Matrix3f, Matrix4f
from lmath import Matrix2d, Matrix3d, Matrix4d

%matplotlib inline

## Rotation matrices (2d)

In [ ]:
w_angle = widgets.FloatSlider(0, min=-np.pi, max=np.pi, step=0.01)
display(w_angle)

In [ ]:
# Have a circle at hand to be drawn when needed :)
tt = np.linspace(-np.pi, np.pi, 100)
xx, yy = np.cos(tt), np.sin(tt)

In [ ]:
mat = Matrix2f.Rotation(w_angle.value)
vec = Vector2f(1.0, 0.0) # x unit vector
vec_rotated = mat * vec
plt.plot(xx, yy, 'b-')
plt.plot([vec.x], [vec.y], 'bo')
plt.plot([vec_rotated.x], [vec_rotated.y], 'ro')
plt.axis([-2, 2, -2, 2])
plt.grid(True)
plt.annotate("original", xy=vec)
plt.annotate("rotated", xy=vec_rotated)

## Scale matrices (2d)

In [ ]:
w_scale = widgets.FloatSlider(1.0, min=1.0, max=5.0, step=0.01)
display(w_scale)

In [ ]:
scale = w_scale.value
mat = Matrix2f.Scale(scale, scale)
vec = Vector2f(1.0, 0.0) # x unit vector
vec_scaled = mat * vec
plt.plot(xx, yy, 'b-')
plt.plot([vec.x], [vec.y], 'bo')
plt.plot([vec_scaled.x], [vec_scaled.y], 'ro')
plt.axis([-5, 5, -5, 5])
plt.grid(True)
plt.annotate("original", xy=vec)
plt.annotate("scaled", xy=vec_scaled)

## Rotation matrices (3d)

In [ ]:
from mpl_toolkits import mplot3d
w_angle_x = widgets.FloatSlider(np.pi/4, min=-np.pi, max=np.pi, step=0.01)
w_angle_y = widgets.FloatSlider(np.pi/4, min=-np.pi, max=np.pi, step=0.01)
w_angle_z = widgets.FloatSlider(np.pi/4, min=-np.pi, max=np.pi, step=0.01)
display(w_angle_x)
display(w_angle_y)
display(w_angle_z)

In [ ]:
# Have a few circles at hand to be drawn when needed :)
n_samples = 100
tt = np.linspace(-np.pi, np.pi, n_samples)
zeros = np.zeros(n_samples)
xx1, yy1 = np.cos(tt), np.sin(tt)
yy2, zz2 = np.cos(tt), np.sin(tt)
zz3, xx3 = np.cos(tt), np.sin(tt)

In [ ]:
rotmat_x = Matrix3f.RotationX(w_angle_x.value)
rotmat_y = Matrix3f.RotationY(w_angle_y.value)
rotmat_z = Matrix3f.RotationZ(w_angle_z.value)

vec_rotx = rotmat_x * Vector3f(0.0, 1.0, 0.0)
vec_roty = rotmat_y * Vector3f(0.0, 0.0, 1.0)
vec_rotz = rotmat_z * Vector3f(1.0, 0.0, 0.0)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(xx1, yy1, zeros)
ax.plot3D(zeros, yy2, zz2)
ax.plot3D(xx3, zeros, zz3)
ax.scatter3D(vec_rotx.x, vec_rotx.y, vec_rotx.z, 'ro')
ax.scatter3D(vec_roty.x, vec_roty.y, vec_roty.z, 'ro')
ax.scatter3D(vec_rotz.x, vec_rotz.y, vec_rotz.z, 'ro')

## Create a 3d visualizer

In [39]:
import os
import time
import numpy as np
from lmath import Vector3d, Vector4d
from lmath import Matrix3d, Matrix4d

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [40]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [41]:
red_material = g.MeshPhongMaterial(color=0xff0000, reflectivity=0.8)
green_material = g.MeshPhongMaterial(color=0x00ff00, reflectivity=0.8)
blue_material = g.MeshPhongMaterial(color=0x0000ff, reflectivity=0.8)
gray_material = g.MeshPhongMaterial(color=0x888888, reflectivity=0.8)

frame = vis["frame"]
frame.delete()
frame["x_axis"].set_object(g.Cylinder(1.0, 0.1), red_material)
frame["x_axis"].set_transform(tf.euler_matrix(0.0, 0.0, np.pi / 2) @ tf.translation_matrix([0.0, -0.5, 0.0]))
frame["y_axis"].set_object(g.Cylinder(1.0, 0.1), green_material)
frame["y_axis"].set_transform(tf.translation_matrix([0.0, 0.5, 0.0]))
frame["z_axis"].set_object(g.Cylinder(1.0, 0.1), blue_material)
frame["z_axis"].set_transform(tf.euler_matrix(np.pi / 2, 0.0, 0.0) @ tf.translation_matrix([0.0, 0.5, 0.0]))
frame["center"].set_object(g.Sphere(0.2), gray_material)

### Note:

* Notice that we're using one extra transposition, because our buffers are stored in column major order, unlike numpy default behaviour that is row-major order
* Notice also that we're using the format `w-x-y-z` for quaternions, whereas meshcat expects the THREE.js format `x-y-z-w`. To avoid issues, we transform our quaternions to that format

In [42]:
def rotation_matrix_to_meshcat_format(matrix):
    return matrix.T

def quaternion_to_meshcat_format(quaternion):
    return np.array([quaternion.x, quaternion.y, quaternion.z, quaternion.w], np.float64)

In [43]:
# rotate the frame around the x-axis
for t in np.linspace(-np.pi, np.pi, 200):
    ### frame.set_transform(tf.rotation_matrix(t + np.pi, [1, 0, 0]))
    frame.set_transform(rotation_matrix_to_meshcat_format(Matrix4d.RotationX(t + np.pi)))
    time.sleep(0.01)

In [44]:
# rotate the frame around the y-axis
for t in np.linspace(-np.pi, np.pi, 200):
    ### frame.set_transform(tf.rotation_matrix(t + np.pi, [0, 1, 0]))
    frame.set_transform(rotation_matrix_to_meshcat_format(Matrix4d.RotationY(t + np.pi).T))
    time.sleep(0.01)

In [45]:
# rotate the frame around the z-axis
for t in np.linspace(-np.pi, np.pi, 200):
    ### frame.set_transform(tf.rotation_matrix(t + np.pi, [0, 0, 1]))
    frame.set_transform(rotation_matrix_to_meshcat_format(Matrix4d.RotationZ(t + np.pi).T))
    time.sleep(0.01)